<a href="https://colab.research.google.com/github/cardazuluaga/DLCourse/blob/main/Ej8LSTMvsGRUPrediccionSeriesTemporales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import GRU, LSTM, Dense
from tensorflow.keras.models import Sequential

# Generar una serie temporal sintética
np.random.seed(0)
timesteps = np.arange(0, 100, 0.1)
seno = np.sin(timesteps)
serie_temporal = seno + np.random.normal(scale=0.1, size=len(timesteps))

# Dividir la serie temporal en conjuntos de entrenamiento y prueba
split_ratio = 0.9
train_size = int(split_ratio * len(timesteps))
train_series = serie_temporal[:train_size]
test_series = serie_temporal[train_size:]

# Función para preparar los datos en secuencias para el entrenamiento
def prepare_data(series, window_size):
    X, y = [], []
    for i in range(len(series) - window_size):
        X.append(series[i:i + window_size])
        y.append(series[i + window_size])
    return np.array(X), np.array(y)

window_size = 10
X_train, y_train = prepare_data(train_series, window_size)
X_test, y_test = prepare_data(test_series, window_size)

model_RNN = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(64, activation='relu', return_sequences=True, input_shape=(window_size, 1)),
    tf.keras.layers.SimpleRNN(32, activation='relu', return_sequences=True),
    tf.keras.layers.SimpleRNN(32, activation='relu'),
    tf.keras.layers.Dense(1)  # Una sola salida para la predicción
])

# Construye el modelo MLP
model_MLP = Sequential([
    Dense(64, activation='relu', input_shape=(window_size,)),
    Dense(32, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)
])

# Crear y entrenar un modelo LSTM
model_lstm = Sequential([
    LSTM(64, input_shape=(window_size, 1)),
    Dense(1)
])

model_gru = Sequential([
    GRU(64, input_shape=(window_size, 1)),
    Dense(1)
])

model_gru.compile(loss='mean_squared_error', optimizer='adam')
model_gru.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

model_lstm.compile(loss='mean_squared_error', optimizer='adam')
model_lstm.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

model_RNN.compile(loss='mean_squared_error', optimizer='adam')
model_RNN.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

model_MLP.compile(loss='mean_squared_error', optimizer='adam')
model_MLP.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

# Evaluar el rendimiento de los modelos en el conjunto de prueba
mse_RNN = model_RNN.evaluate(X_test, y_test)
mse_MLP = model_MLP.evaluate(X_test, y_test)
mse_lstm = model_lstm.evaluate(X_test, y_test)
mse_gru = model_gru.evaluate(X_test, y_test)

print("Error cuadrático medio (MLP):", mse_MLP)
print("Error cuadrático medio (RNN):", mse_RNN)
print("Error cuadrático medio (LSTM):", mse_lstm)
print("Error cuadrático medio (GRU):", mse_gru)

# Hacer predicciones
predictions_RNN = model_RNN.predict(X_test)
predictions_MLP = model_MLP.predict(X_test)
predictions_lstm = model_lstm.predict(X_test)
predictions_gru = model_gru.predict(X_test)

In [ ]:
# Visualizar las predicciones
plt.figure(figsize=(12, 6))
plt.plot(timesteps[train_size + window_size:], y_test, label='Datos reales', color='blue')
plt.plot(timesteps[train_size + window_size:], predictions_MLP, label='Predicciones MLP', color='green')
plt.plot(timesteps[train_size + window_size:], predictions_RNN, label='Predicciones RNN', color='cyan')
plt.plot(timesteps[train_size + window_size:], predictions_lstm, label='Predicciones LSTM', color='red')
plt.plot(timesteps[train_size + window_size:], predictions_gru, label='Predicciones GRU', color='magenta')
plt.legend()
plt.title('Comparación de predicciones GRU vs. LSTM')
plt.xlabel('Tiempo')
plt.ylabel('Valor')
plt.show()